In [110]:

from pandas import  *
from numpy import *
from seaborn import *
from matplotlib.pyplot import *
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
import string


In [111]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(50)

# Number of records
n_records = 100000

# Generate random data for each feature
sound_level = np.random.normal(80, 30, n_records).astype(int)  # dcb, with some values reaching up to 150 dcb
temperature = np.random.normal(65, 15, n_records).astype(int)  # Celsius
battery_drain = np.random.choice([0, 1], size=n_records, p=[0.85, 0.15])  # 0 (normal) or 1 (draining fast)
process_halted = np.random.choice([0, 1], size=n_records, p=[0.95, 0.05])  # 0 (normal) or 1 (error halted)

# Define a more complex set of causes based on detailed conditions
cause = [
    "Oil Pressure Drop" if sl < 50 and temp < 50 and battery == 0 else
    "Overload Error" if sl > 130 and temp > 90 and halted == 1 else
    "Power Surge Alert" if battery == 1 and temp > 85 and 90 <= sl < 130 else
    "Vibration and Heat Spike" if 95 <= sl < 115 and temp > 80 else
    "Critical System Fault" if halted == 1 and temp > 90 and sl > 125 else
    "System Cooling Failure" if battery == 0 and temp >= 85 and sl < 90 else
    "Sensor Calibration Required" if (50 <= sl < 65 or 45 <= temp < 60) and battery == 1 else
    "Mechanical Fatigue Warning" if (sl > 110 and temp < 60) or (temp > 70 and halted == 1) else
    "Processor Overload" if battery == 1 and halted == 1 and temp < 75 else
    "Excessive Battery Drain" if battery == 1 and 60 <= temp < 80 else
    "Warning: High Temp Fluctuations" if np.abs(temp - np.mean(temperature)) > 20 else
    "Unstable Power Detected" if battery == 1 and sl < 70 and np.random.rand() < 0.05 else
    "Critical Shutdown Warning" if halted == 1 and sl > 130 else
    "Abnormal Sound Pattern" if sl >= 100 and sl <= 120 and np.random.rand() < 0.03 else
    "Sudden System Halt" if halted == 1 and sl < 70 else
    "Temperature Anomaly Detected" if temp > 75 and temp < 85 and np.random.rand() < 0.02 else
    "Frequent Battery Drain" if battery == 1 and np.random.rand() < 0.1 else
    "Energy Leakage" if battery == 1 and sl > 90 and np.random.rand() < 0.07 else
    "Unknown Error Code 37" if np.random.rand() < 0.01 else
    "Normal Operation"
    for sl, temp, battery, halted in zip(sound_level, temperature, battery_drain, process_halted)
]

# Create DataFrame
df = pd.DataFrame({
    "Sound_Level_dcb": sound_level,
    "Temperature_C": temperature,
    "Battery_Drain": battery_drain,
    "Process_Halted": process_halted,
    "Cause": cause
})

# Display the first few rows
df.head()


,Sound_Level_dcb,Temperature_C,Battery_Drain,Process_Halted,Cause
0,33,72,0,0,Normal Operation
1,79,73,0,0,Normal Operation
2,61,58,0,0,Normal Operation
3,36,74,0,0,Normal Operation
4,122,46,0,0,Mechanical Fatigue Warning


In [112]:
df.shape

(100000, 5)

In [113]:
x=df.iloc[:,:-1]

In [114]:
x

,Sound_Level_dcb,Temperature_C,Battery_Drain,Process_Halted
0,33,72,0,0
1,79,73,0,0
2,61,58,0,0
3,36,74,0,0
4,122,46,0,0
...,...,...,...,...
99995,58,84,0,0
99996,71,89,0,0
99997,84,57,0,0
99998,93,61,0,0


In [115]:
y=df.iloc[:,-1:]

In [116]:
y

,Cause
0,Normal Operation
1,Normal Operation
2,Normal Operation
3,Normal Operation
4,Mechanical Fatigue Warning
...,...
99995,Normal Operation
99996,System Cooling Failure
99997,Normal Operation
99998,Normal Operation


In [117]:
df["Cause"].unique()


array(['Normal Operation', 'Mechanical Fatigue Warning',
       'Abnormal Sound Pattern', 'Warning: High Temp Fluctuations',
       'Vibration and Heat Spike', 'Sensor Calibration Required',
       'Excessive Battery Drain', 'System Cooling Failure',
       'Oil Pressure Drop', 'Unknown Error Code 37', 'Sudden System Halt',
       'Power Surge Alert', 'Critical System Fault',
       'Unstable Power Detected', 'Processor Overload',
       'Temperature Anomaly Detected', 'Frequent Battery Drain',
       'Critical Shutdown Warning', 'Energy Leakage', 'Overload Error'],
      dtype=object)

In [118]:

# Step 3: Train the Random Forest Classifier
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [119]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

c:\Users\debna\anaconda3\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [120]:
model.score(X_train, y_train)

0.98855

In [121]:
model.score(X_test, y_test)

0.9869

In [122]:
model.predict([[99,	79	,1,	1]])

c:\Users\debna\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array(['Mechanical Fatigue Warning'], dtype=object)